# HW 3 Classification
28 Feb 2025



## Links to Resources
- Natural and False Color Composites - https://gsp.humboldt.edu/olm/Courses/GSP_216/lessons/composites.html
- About Landsat 8 - https://docs.sentinel-hub.com/api/latest/data/landsat-8/ 
- Supervised Classification Tutorial - https://www.youtube.com/watch?v=aHaatFbmZFw&ab_channel=SpatialeLearning 
- Classification Calculation - https://www.linkedin.com/pulse/ndvi-ndbi-ndwi-calculation-using-landsat-7-8-tek-bahadur-kshetri/ 


In [1]:
import ee
import geemap
import matplotlib.pyplot as plt
import numpy as np

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-wongluyii')

### Landsat 8 Image Selection

In [22]:
# Modified Version 2
# Pulling Phildelphia boundary (Region of Interest)
philly = ee.FeatureCollection("TIGER/2018/Counties").filter(ee.Filter.eq("NAME", "Philadelphia"))

# Apply scaling factor 
def applyScaleFactor(landsat):
  opticalBands = landsat.select("SR_B.").multiply(0.0000275).add(-0.2)
  return landsat.addBands(opticalBands, None, True)

# Load Landsat 8 Collection 
# Might have to change this so that it is pulling only one date
landsat = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
           .filterDate("2023-01-01", "2023-12-31")
           .filterBounds(philly)
           .map(applyScaleFactor)
           .filterMetadata("CLOUD_COVER", "less_than", 10)
           .median()
           .clip(philly))


stats = landsat.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=philly,
    scale=30
)

# ## MAYBE MOVE DOWN 
# ndvi = landsat.normalizedDifference(["SR_B5", "SR_B4"]).rename("ndvi")
# ndbi = landsat.normalizedDifference(["SR_B6", "SR_B5"]).rename("ndbi")
# mndwi = landsat.normalizedDifference(["SR_B3", "SR_B6"]).rename("mndwi")

# images = landsat.addBands(ndvi).addBands(ndbi).addBands(mndwi)

visualization = {
  "bands": ["SR_B4", "SR_B3", "SR_B2"],
  "min": 0.0,
  "max": 0.3
}

urban_viz = {
  "bands": ["SR_B7", "SR_B4", "SR_B6"],
  "min": 0.0,
  "max": 0.3
}

veg_viz = {
  "bands": ["SR_B5", "SR_B4", "SR_B3"],
  "min": 0.0,
  "max": 0.3
}

# Visualize ROI and Landsat image
Map = geemap.Map()
Map.centerObject(philly, 10)
Map.addLayer(landsat, visualization, "True Color (432)")
Map.addLayer(landsat, urban_viz, "False Color (746)")
Map.addLayer(landsat, veg_viz, "False Color (543)")

# min_value = stats.get("B4_min")
# max_value = stats.get("B4_max")
# max_false = stats.get("B7_max")
# max_veg = stats.get("B5_max")
# Map.addLayer(landsat, {"bands": ["B4", "B3", "B2"], "min": 5, "max": max_value}, "True Color")
# Map.addLayer(landsat, {"bands": ["B7", "B4", "B6"], "min": 5, "max": max_false}, "False Color (Urban)")
# Map.addLayer(landsat, {"bands": ["B5", "B4", "B3"], "min": 5, "max": max_veg}, "False Color (Veg)")
Map

Map(center=[40.00760439369441, -75.13403899470623], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# Map.addLayer(landsat, {"bands": ["B5", "B4", "B3"], "min": 5, "max": max_veg}, "False Color (Veg)")
# Define a boundary
small_area = philly.geometry().buffer(10)
# Export False color (Veg)
geemap.ee_export_image(landsat,  scale=30, filename="landsat_philly_2023_v2.tif", region=small_area)

In [18]:
# Check
print("The Image Date is:", ee.Date(landsat.get("system:time_start")).format("YYYY-MM-dd").getInfo())

EEException: Date: Parameter 'value' is required and may not be null.

## Pull in Sample Data 

In [4]:
# Pull in manually labeled data from QGIS
training_samples = geemap.geojson_to_ee("/Users/luyiiwong/Documents/GitHub/musa-650-spring-2025/assignments/samples.geojson")

In [5]:
# Merge training samples based on class 
urban = training_samples.filter(ee.Filter.eq("class", "urban"))
bare = training_samples.filter(ee.Filter.eq("class", "bare"))
water = training_samples.filter(ee.Filter.eq("class", "water"))
vegetation = training_samples.filter(ee.Filter.eq("class", "vegetation"))

# Merge all labeled samples into a single dataset
sample = urban.merge(bare).merge(water).merge(vegetation)

In [6]:
# Update classifiers into a numeric labels - create label mapping
label_map = ee.Dictionary({
    "urban": 0,
    "bare": 1,
    "water": 2,
    "vegetation": 3
})

# Function to assign numeric labels
def add_numeric_label(feature):
    return feature.set("landcover", label_map.get(feature.get("class")))

# Apply function to training data
sample = sample.map(add_numeric_label)


## Feature Engineering

The Normal Difference Vegetation Index (NDVI), Normal Difference Built-up Index (NDBI), and MNDWI (Modified Normalized Difference Water Index) can be calculated using the following:
- NDVI = (B5 - B4)/(B5 + B4)
- NDBI = (B6 - B5)/(B6 + B5)
- MNDWI = (B3 - B6)/(B3 + B6)

### T1 Version

In [7]:
# Use normalizedDifference to compute (first − second) / (first + second)
# Calculate NDVI
ndvi = landsat.normalizedDifference(["B5", "B4"]).rename("NDVI")

# Calculate NDBI
ndbi = landsat.normalizedDifference(["B6", "B5"]).rename("NDBI")

# Calculate MNDWI
mndwi = landsat.normalizedDifference(["B3", "B6"]).rename("MNDWI")

In [17]:
# Load and normalize elevation data
dem = ee.Image("USGS/SRTMGL1_003").clip(philly)
slope = ee.Terrain.slope(dem)

# Normalize bands to 0-1 scale
def normalize(image, band):
    min_max = image.reduceRegion(ee.Reducer.minMax(), philly, 30)
    min_val = ee.Number(min_max.get(f"{band}_min"))
    max_val = ee.Number(min_max.get(f"{band}_max"))
    return image.select(band).subtract(min_val).divide(max_val.subtract(min_val)).rename(f"{band}_norm")

bands_to_normalize = ["B1", "B2", "B3", "B4", "B5", "B6", "B7"]
normalized_bands = [normalize(landsat, band) for band in bands_to_normalize]

# Combine all features
final_image = landsat.addBands([ndvi, ndbi, mndwi, slope] + normalized_bands)

### T1_L2 Version

In [8]:
## IF USING T1_L2
# Use normalizedDifference to compute (first − second) / (first + second)
ndvi = landsat.normalizedDifference(["SR_B5", "SR_B4"]).rename("ndvi")
ndbi = landsat.normalizedDifference(["SR_B6", "SR_B5"]).rename("ndbi")
mndwi = landsat.normalizedDifference(["SR_B3", "SR_B6"]).rename("mndwi")

In [23]:
## IF USING T1_L2
# Load and normalize elevation data
# Load 30m DEM dataset
dem = ee.Image("USGS/SRTMGL1_003").clip(philly)
slope = ee.Terrain.slope(dem)

# Normalize bands to 0-1 scale
def normalize(image, band):
    min_max = image.reduceRegion(ee.Reducer.minMax(), philly, 30)
    min_val = ee.Number(min_max.get(f"{band}_min"))
    max_val = ee.Number(min_max.get(f"{band}_max"))
    return image.select(band).subtract(min_val).divide(max_val.subtract(min_val)).rename(f"{band}_norm")

bands_to_normalize = ["B1", "B2", "B3", "B4", "B5", "B6", "B7"]
normalized_bands = [normalize(landsat, band) for band in bands_to_normalize]

# Combine all features
final_image = landsat.addBands([ndvi, ndbi, mndwi, slope] + normalized_bands)

In [24]:
# Load and visualize SRTM DEM
dem = ee.Image("USGS/SRTMGL1_003").clip(philly)
dem_vis = {"min": 0, "max": 500, "palette": ["black", "A9A9A9", "D3D3D3", "white"]}

Map.addLayer(dem, dem_vis, "Elevation (SRTM DEM)")

In [15]:
# Combine NDVI, NDBI, MNDWI, and Slope to samples 
# Sample pixel values from final_image at training sample locations
sample = final_image.sampleRegions(
    collection=sample,  
    properties=["landcover"],  
    scale=30,  
    geometries=True,  
    maxPixels=2e9 
)

## UNABLE TO RUN: Too many pixels in the region

# Check that features were added
print(sample.first().getInfo())

TypeError: Image.sampleRegions() got an unexpected keyword argument 'maxPixels'

## Model Training and Evaluation

### Training

In [27]:
# Split Training and Test Sets
# Split: 70/30
sample = sample.randomColumn("split")
train_data = sample.filter(ee.Filter.lt("split", 0.7))
test_data = sample.filter(ee.Filter.gte("split", 0.7))

In [29]:
# Feature List
# Define predictor variables (features) used for training
feature_list = ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7", "ndvi", "ndbi", "mndwi", "slope"]


In [31]:
print(train_data.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-75.17123688993095, 39.93049329131763]}, 'id': '1_1_1_100', 'properties': {'class': 'urban', 'landcover': 0, 'split': 0.6425343829846653}}


In [28]:
# Train Random Forest Model
rf_model = ee.Classifier.smileRandomForest(100).train(
    features=train_data,  
    classProperty="landcover",  
    inputProperties=feature_list  
)

NameError: name 'feature_list' is not defined

In [20]:
# Extract Importance Scores
importance = rf_model.explain().get("importance")
print(importance.getInfo())  # Outputs feature importance scores


### Model Evaluation

In [ ]:
# Accuracy Assessment 
# Apply trained model to Landsat 8 image
classified = final_image.classify(rf_model)

In [ ]:
Map.addLayer(classified, {"min": 0, "max": 3, "palette": ["gray", "brown", "blue", "green"]}, "Land Cover Classification")
Map

In [ ]:
# Model Performance
confusion_matrix = rf_model.confusionMatrix()
print(confusion_matrix.getInfo()) 

In [ ]:
overall_acc = confusion_matrix.accuracy()
precision = confusion_matrix.producersAccuracy()  # Class-wise
recall = confusion_matrix.consumersAccuracy()  # Class-wise
print("Overall Accuracy:", overall_acc.getInfo())
print("Precision:", precision.getInfo())
print("Recall:", recall.getInfo())

In [ ]:
# ESA Comparison 
# Do Visual Comparison
esa_landcover = ee.ImageCollection("ESA/WorldCover/v200").first().clip(philly)
Map.addLayer(esa_landcover, {"min": 10, "max": 100, "palette": ["red", "blue", "green", "yellow"]}, "ESA WorldCover")

## Results

In [21]:
# Export Landcover as GeoTIFF
export_task = ee.batch.Export.image.toDrive(
    image=classified,
    description="LandCover_Classification",
    scale=30,
    region=philly.geometry(),
    fileFormat="GeoTIFF"
)
export_task.start()

# Export confusion metrix and accuracy as CSV
confusion_matrix_array = ee.Array(confusion_matrix)
export_task2 = ee.batch.Export.table.toDrive(
    collection=ee.FeatureCollection([
        ee.Feature(None, {"confusion_matrix": confusion_matrix_array})
    ]),
    description="ConfusionMatrix",
    fileFormat="CSV"
)
export_task2.start()



# OLD/DISCARD 
## Check Image Properties

In [ ]:
# Old Version
# Pulling Phildelphia boundary (Region of Interest)
philly = ee.FeatureCollection("TIGER/2018/Counties").filter(ee.Filter.eq("NAME", "Philadelphia"))

# Load Landsat 8 Collection 
landsat = (ee.ImageCollection("LANDSAT/LC08/C02/T1")
           .filterBounds(philly)
           .filterDate("2023-01-01", "2023-12-31")
           .filterMetadata("CLOUD_COVER", "less_than", 10)
           .median()
           .clip(philly)
           .select("B[1-7]"))


# Visualize ROI and Landsat image
Map = geemap.Map()
Map.centerObject(philly, 10)

stats = landsat.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=philly,
    scale=30
)
min_value = stats.get("B4_min")
max_value = stats.get("B4_max")
max_false = stats.get("B7_max")
max_veg = stats.get("B5_max")
Map.addLayer(landsat, {"bands": ["B4", "B3", "B2"], "min": 5, "max": max_value}, "True Color")
Map.addLayer(landsat, {"bands": ["B7", "B4", "B6"], "min": 5, "max": max_false}, "False Color (Urban)")
Map.addLayer(landsat, {"bands": ["B5", "B4", "B3"], "min": 5, "max": max_veg}, "False Color (Veg)")
Map

In [18]:
print("The Image Date is:", ee.Date(image.get("system:time_start")).format("YYYY-MM-dd").getInfo())
print("The cloud cover is:", image.get("CLOUD_COVER").getInfo())

The Image Date is: 2023-04-10
The cloud cover is: 0.08
